<a href="https://colab.research.google.com/github/NikolValerie/homework/blob/main/ruBERT_tiny2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from google.colab import files
uploaded = files.upload()

Saving tg_channels.zip to tg_channels.zip


In [4]:
import zipfile
import io

file_name = 'tg_channels.zip'

with zipfile.ZipFile(io.BytesIO(uploaded[file_name]), 'r') as zip_ref:
    zip_ref.extractall('/content')

In [5]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch

import matplotlib.pyplot as plt

# импортируем трансформеры
import transformers
import warnings
warnings.filterwarnings('ignore')

In [6]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cpu


In [15]:
df = pd.read_csv('/content/tg_channels/train_channels.csv', encoding='UTF-8')
df

,date,content,label,initial_text
0,2023-07-05 15:16:45+00:00,Две ссылкиРас: https://t.me/PlushevChannel/224...,технологии,Две ссылкиРас: https://t.me/PlushevChannel/224...
1,2023-07-11 13:13:02+00:00,"@firstartclub | #обзор«Отец», Константин Аполл...",искусство,"@firstartclub | #обзор«Отец», Константин Аполл..."
2,2022-12-17 07:55:28+00:00,"pov: когда объясняют новую тему урока, а ты ещ...",образование_познавательное,"pov: когда объясняют новую тему урока, а ты ещ..."
3,2023-05-25 10:37:54+00:00,⚡️Грузинская авиакомпания Georgian Airways зап...,маркетинг,⚡️Грузинская авиакомпания Georgian Airways зап...
4,2020-08-03 10:01:15+00:00,"Интересуешься историей и людьми, которые ее со...",образование_познавательное,"Интересуешься историей и людьми, которые ее со..."
...,...,...,...,...
24799,2023-09-10 11:07:37+00:00,✅ groza.media не заблокирован 🔥 но может быть...,технологии,✅ groza.media не заблокирован 🔥 но может быть...
24800,2023-01-08 13:51:54+00:00,"Друзья, я сейчас в Тбилиси!Было бы здорово соб...",технологии,"Друзья, я сейчас в Тбилиси!Было бы здорово соб..."
24801,2021-02-24 09:41:28+00:00,Без лишних слов — Школа менеджеров Яндекса 202...,технологии,Без лишних слов — Школа менеджеров Яндекса 202...
24802,2023-05-22 14:42:58+00:00,"И еще один крутой пример ""AIaaS"" (""ИИ как услу...",технологии,"И еще один крутой пример ""AIaaS"" (""ИИ как услу..."


In [81]:
# возьмем только первые сто ради экономии времени
df_red = df[['content', 'label']].iloc[:10000, :]

In [82]:
# кодируем класс
df_red['sen_enc'] = LabelEncoder().fit_transform(df_red['label'])

In [83]:
df_red.head()

,content,label,sen_enc
0,Две ссылкиРас: https://t.me/PlushevChannel/224...,технологии,4
1,"@firstartclub | #обзор«Отец», Константин Аполл...",искусство,1
2,"pov: когда объясняют новую тему урока, а ты ещ...",образование_познавательное,3
3,⚡️Грузинская авиакомпания Georgian Airways зап...,маркетинг,2
4,"Интересуешься историей и людьми, которые ее со...",образование_познавательное,3


Загружаем модель

In [12]:
# rubert-tiny2:
from transformers import AutoTokenizer, AutoModel

## задаем саму модель
model = AutoModel.from_pretrained("cointegrated/rubert-tiny2")

# токенайзер к ней (для некоторых моделей токенайзер будет отличаться, см.
## в документации к каждой модели конкретно)
tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny2")



config.json:   0%|          | 0.00/693 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/118M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/401 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.74M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [13]:
model

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(83828, 312, padding_idx=0)
    (position_embeddings): Embedding(2048, 312)
    (token_type_embeddings): Embedding(2, 312)
    (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-2): 3 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=312, out_features=312, bias=True)
            (key): Linear(in_features=312, out_features=312, bias=True)
            (value): Linear(in_features=312, out_features=312, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=312, out_features=312, bias=True)
            (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
   

In [84]:
## зададим максимальную длину последовательности
MAX_LEN = 128

In [85]:
# токенизируем одну фразу
tokenizer.encode(
    'Привет как дела',
    add_special_tokens=True,
    truncation=True,
    max_length=MAX_LEN,
    # padding='max_length'
    )

[2, 51343, 1150, 7843, 3]

In [86]:
tokenizer.decode(51343)

'Привет'

In [87]:
# применяем токенизатор:
# -≥ add_special_tokens = добавляем служебные токены (CLS=2, SEP=3)
# -≥ truncation = обрезаем по максимальной длине
# -≥ max_length = максимальная длина последовательности
tokenized = df_red['content'].apply((lambda x: tokenizer.encode(x,
                                                                      add_special_tokens=True,
                                                                      truncation=True,
                                                                      max_length=MAX_LEN)))

In [88]:
tokenized

0       [2, 23706, 35999, 4845, 4992, 30, 2986, 30, 19...
1       [2, 36, 804, 3623, 11205, 1, 7, 33238, 105, 18...
2       [2, 867, 846, 30, 3570, 48888, 18856, 18377, 4...
3       [2, 1, 46655, 15682, 9451, 68723, 34168, 38353...
4       [2, 32305, 644, 35256, 36666, 320, 21667, 16, ...
                              ...                        
9995    [2, 10721, 23101, 37121, 19319, 29726, 320, 33...
9996    [2, 33488, 56616, 21494, 35, 299, 29642, 32000...
9997    [2, 35129, 2386, 33541, 68955, 16, 4146, 30903...
9998    [2, 25202, 26075, 55861, 15439, 18022, 650, 16...
9999    [2, 6226, 35531, 46605, 35855, 30598, 2677, 16...
Name: content, Length: 10000, dtype: object

In [89]:
# посмотрим на результат токенизатора
# 2 - CLS – classification token
# 3 – EOS – end of sequence
tokenized.values[:2]

array([list([2, 23706, 35999, 4845, 4992, 30, 2986, 30, 19, 19, 87, 18, 1127, 19, 5414, 1607, 846, 1097, 1694, 4669, 19, 6451, 1140, 1290, 8759, 2112, 30, 2986, 30, 19, 19, 859, 18, 85, 21332, 18, 2643, 19, 5646, 67, 537, 67, 2120, 19, 1078, 19, 1107, 19, 42271, 19, 1375, 539, 1290, 24339, 15582, 1433, 539, 1383, 1433, 29506, 5864, 1320, 587, 1140, 1320, 25452, 1012, 831, 9722, 656, 18, 18, 18, 280, 10030, 10632, 8341, 22933, 17, 1619, 5277, 18, 18, 18, 3]),
       list([2, 36, 804, 3623, 11205, 1, 7, 33238, 105, 18876, 117, 16, 10849, 79003, 3190, 46822, 68765, 12, 2139, 13, 39251, 5382, 20113, 17, 105, 297, 42435, 36789, 5189, 66687, 1229, 32770, 1373, 117, 18, 34196, 14357, 16, 1150, 78173, 22326, 5021, 2182, 16, 1908, 320, 2445, 51487, 4427, 16, 34913, 54586, 42210, 18, 49614, 16, 39329, 775, 38368, 66692, 656, 778, 40191, 78668, 26106, 16, 16296, 15899, 679, 320, 48109, 18, 5971, 28910, 32007, 16, 1363, 46127, 36801, 17, 33145, 48697, 329, 43663, 18, 3280, 3687, 769, 40330, 44713,

In [90]:
# проверим длины последовательностей: максимальная будет 128,
# но какие-то будут меньше
for i in tokenized.values[:10]:
    print(len(i), end=' ')

85 128 18 64 53 54 87 102 33 128 

In [91]:
# чтобы дополнить последовательности до фиксированной длины, допишем нули к
# коротким последовательностям
padded = np.array([i + [0]*(MAX_LEN-len(i)) for i in tokenized.values])

In [92]:
# проверим
# теперь все последовательности должны быть одинаковой длины - max_len
for i in padded[:10]:
    print(len(i), end=' ')

128 128 128 128 128 128 128 128 128 128 

In [93]:
padded[78]

array([    2, 43203,   613, 18708,    17,  2389, 36091,     1,     7,
       36214,   733, 13253, 10565,  2597,   666, 73198, 43057, 42680,
       27228,   548, 32079,    16, 11352,    17,  1619, 25913, 39596,
        1588, 83198, 39832,    16,   312, 30411,    17,  1619, 30440,
       34529, 17950,  7757, 29292,    18,   282,  3200, 71587, 26629,
       49540,    16, 29820, 38090, 25473,  2608, 36091,   761,    16,
       29860,   548,  1619, 29420, 36462,   320,  1150,  1908, 33642,
          16,  1046,   314,  5557, 36091, 42504,  9977, 49552,  9022,
         705, 23729, 18232,    18, 36303, 27662, 36214,    30,  2986,
          30,    19,    19,  2406,  1173,    18,   747,    19,    71,
        7747,  1469,    67,    79,  1140,  1429,  1478,   559,     3,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0]

Masking

In [94]:
# чтобы attention не считался для позиций, где установлены фиктивные
# нулевые токены, мы делаем для него маску
attention_mask = np.where(padded != 0, 1, 0)
attention_mask[78]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [95]:
print(padded.shape); print(attention_mask.shape)

(10000, 128)
(10000, 128)


Сделаем датасет из наших массивов (это необязательно, но желательно).

In [96]:
class BertInputs(torch.utils.data.Dataset):
    def __init__(self, tokenized_inputs, attention_masks):
        super().__init__()
        self.tokenized_inputs = tokenized_inputs
        self.attention_masks = attention_masks

    def __len__(self):
        return self.tokenized_inputs.shape[0]

    def __getitem__(self, idx):
        ids = self.tokenized_inputs[idx]
        ams = self.attention_masks[idx]

        return ids, ams

dataset = BertInputs(padded, attention_mask)

In [97]:
loader = torch.utils.data.DataLoader(dataset, batch_size=10, shuffle=False)
sample_ids, sample_ams = next(iter(loader))
print(sample_ids.shape, sample_ams.shape)

torch.Size([10, 128]) torch.Size([10, 128])


In [98]:
model.to(device)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(83828, 312, padding_idx=0)
    (position_embeddings): Embedding(2048, 312)
    (token_type_embeddings): Embedding(2, 312)
    (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-2): 3 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=312, out_features=312, bias=True)
            (key): Linear(in_features=312, out_features=312, bias=True)
            (value): Linear(in_features=312, out_features=312, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=312, out_features=312, bias=True)
            (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
   

In [99]:
%%time
features = []
for inputs, attention_masks in loader:
# Получаем выход модели (нам оттуда нужно не все)
    with torch.inference_mode():
        last_hidden_states = model(inputs.cpu(), attention_mask=attention_masks.cpu())
        vectors = last_hidden_states[0][:,0,:].cpu().numpy()
    features.extend(vectors)
len(features)

CPU times: user 15min 22s, sys: 2min 14s, total: 17min 36s
Wall time: 22 s


10000

In [100]:
features[0].shape

(312,)

In [101]:
# Прогоним тестовый батч
with torch.inference_mode():
    last_hs = model(sample_ids.cpu(), attention_mask=sample_ams.cpu())
print(last_hs[0].shape)

torch.Size([10, 128, 312])


In [102]:
model_out_shape = last_hs[0].shape
print(model_out_shape)
print(f'{model_out_shape[0]} - число объектов,\n \
{model_out_shape[1]} - длина каждой последовательности, \n \
{model_out_shape[2]} - длина выходного вектора BERT для одного элемента последовательности')

torch.Size([10, 128, 312])
10 - число объектов,
 128 - длина каждой последовательности, 
 312 - длина выходного вектора BERT для одного элемента последовательности


In [103]:
len(features)

10000

In [104]:
X_train, X_val, y_train, y_val = train_test_split(features, df_red['sen_enc'].iloc[:10000])
print(f'Features shape: {len(X_train)}, Target shape: {len(y_train)}')

Features shape: 7500, Target shape: 7500


In [105]:
df_red['sen_enc'].value_counts()/len(features)*100

sen_enc
4    29.76
2    20.40
0    20.06
3    16.29
1    13.49
Name: count, dtype: float64

In [106]:
from sklearn.linear_model import LogisticRegression

# инициализируем модель логистической регрессии
log_reg = LogisticRegression(max_iter=1000)

# обучаем модель на тренировочных данных
log_reg.fit(X_train, y_train)

# оценка точности модели на валидационных данных
accuracy = log_reg.score(X_val, y_val)
print(f'Accuracy: {accuracy}')

Accuracy: 0.7564


In [107]:
# Выберем случайный текст из датасета
random_idx = np.random.randint(0, len(df_red))
text = df_red['content'].iloc[random_idx]
true_label = df_red['label'].iloc[random_idx]

# Предсказание
pred_label = clf.predict([features[random_idx]])[0]
pred_label_text = df_red['label'].loc[df_red['sen_enc'] == pred_label].iloc[0]

print(f"Текст: {text}")
print(f"Реальный лэйбл: {true_label}")
print(f"Предсказанный лэйбл: {pred_label_text}")

Текст: Когда разобрал сложную тему с Математиком Андреем и идешь на урок в полной ясности 🔥
Реальный лэйбл: образование_познавательное
Предсказанный лэйбл: образование_познавательное


In [112]:
import torch.nn as nn

class CustomBERT(nn.Module):
    def __init__(self, bert_model):
        super(CustomBERT, self).__init__()
        self.bert = bert_model
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(self.bert.config.hidden_size, df_red['sen_enc'].nunique())

    def forward(self, input_ids, attention_mask):
        output = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = output.pooler_output
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        return logits

custom_bert_model = CustomBERT(model)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
custom_bert_model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(custom_bert_model.parameters(), lr=2e-5)

num_epochs = 4
for epoch in range(num_epochs):
    custom_bert_model.train()
    for i, (inputs, attention_masks) in enumerate(loader):
        inputs, attention_masks = inputs.to(device), attention_masks.to(device)
        labels = torch.tensor(df_red['sen_enc'].iloc[:10000].values[i*10:(i+1)*10]).to(device)  # выборка меток только для текущего батча

        optimizer.zero_grad()
        outputs = custom_bert_model(inputs, attention_masks)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        if i % 100 == 0:
            print(f'Epoch {epoch + 1}, Batch {i}, Loss: {loss.item()}')


custom_bert_model.eval()
with torch.no_grad():
    outputs = custom_bert_model(torch.tensor(padded[:10000]).to(device), torch.tensor(attention_mask[:10000]).to(device))
    _, predicted = torch.max(outputs, 1)

accuracy = (predicted.cpu().numpy() == y_val).mean()
print(f'Accuracy on validation set: {accuracy}')

Epoch 1, Batch 0, Loss: 1.7529178857803345
Epoch 1, Batch 100, Loss: 0.6414965391159058
Epoch 1, Batch 200, Loss: 0.4155500829219818
Epoch 1, Batch 300, Loss: 0.24055223166942596
Epoch 1, Batch 400, Loss: 0.1040404811501503
Epoch 1, Batch 500, Loss: 0.10617856681346893
Epoch 1, Batch 600, Loss: 0.13381628692150116
Epoch 1, Batch 700, Loss: 0.04296010360121727
Epoch 1, Batch 800, Loss: 0.07764606922864914
Epoch 1, Batch 900, Loss: 0.30803439021110535
Epoch 2, Batch 0, Loss: 0.22502613067626953
Epoch 2, Batch 100, Loss: 0.015006574802100658
Epoch 2, Batch 200, Loss: 0.017749955877661705
Epoch 2, Batch 300, Loss: 0.024924883618950844
Epoch 2, Batch 400, Loss: 0.03827900439500809
Epoch 2, Batch 500, Loss: 0.023458069190382957
Epoch 2, Batch 600, Loss: 0.018876710906624794
Epoch 2, Batch 700, Loss: 0.01274495292454958
Epoch 2, Batch 800, Loss: 0.015328958630561829
Epoch 2, Batch 900, Loss: 0.1338089108467102
Epoch 3, Batch 0, Loss: 0.01822884939610958
Epoch 3, Batch 100, Loss: 0.00557243032

ValueError: ('Lengths must match to compare', (2500,), (10000,))